In [1]:
%pip install requests beautifulsoup4 lxml
%pip install openpyxl xlsxwriter

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:


BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

def get_max_page(prodi_id):
    """Mendapatkan jumlah halaman maksimal untuk prodi tertentu"""
    try:
        url = f"{BASE_URL}/{prodi_id}/1"
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")
        
        pagination = soup.select('a[href*="byprod"]')
        if pagination:
            last_page = 1
            for link in pagination:
                href = link.get('href', '')
                if f'/byprod/{prodi_id}/' in href:
                    page_match = re.search(rf'/byprod/{prodi_id}/(\d+)', href)
                    if page_match:
                        page_num = int(page_match.group(1))
                        last_page = max(last_page, page_num)
            return last_page
        return 1
    except:
        return 1

def pta():
    """Fungsi utama untuk scraping data PTA"""
    start_time = time.time()
    
    data = {
        "judul": [], "abstrak_id": [], "abstrak_en": []
    }
    
    prodi_id = 7
    prodi_name = "Manajemen"
    
    # Get max pages
    max_page = get_max_page(prodi_id)
    
    for j in range(1, max_page + 1):
        try:
            url = f"{BASE_URL}/{prodi_id}/{j}"
            r = requests.get(url, timeout=15)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')
            
            # Process setiap jurnal
            for jurnal in jurnals:
                try:
                    link_keluar = jurnal.select_one('a.gray.button')['href']
                    
                    response = requests.get(link_keluar, timeout=15)
                    soup1 = BeautifulSoup(response.content, "html.parser")
                    isi = soup1.select_one('div#content_journal')
                    
                    if not isi:
                        continue
                    
                    # Extract data
                    judul = isi.select_one('a.title').text.strip()
                    
                    # Extract abstrak
                    paragraf = isi.select('p[align="justify"]')
                    abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                    abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"
                    
                    # Clean abstrak Indonesia
                    if abstrak_id != "N/A" and abstrak_id.upper().startswith('ABSTRAK'):
                        abstrak_id = abstrak_id[7:].strip()
                    
                    # Append data
                    data["judul"].append(judul)
                    data["abstrak_id"].append(abstrak_id)
                    data["abstrak_en"].append(abstrak_en)
                    
                    time.sleep(0.1)
                    
                except:
                    continue
            
        except:
            continue
    
    # Buat DataFrame
    df = pd.DataFrame(data)
    
    # Summary
    print(f"📊 Total data dikumpulkan: {len(df):,}")
    
    return df

# Jalankan scraping
result_df = pta()

# Ambil abstrak Indonesia untuk corpus
corpus = result_df[result_df['abstrak_id'] != 'N/A']['abstrak_id'].tolist()

# Tampilkan dalam bentuk DataFrame (semua data)
import pandas as pd
df_sample = pd.DataFrame({
    'No': range(1, len(corpus) + 1),
    'Abstrak': corpus
})

df_sample

📊 Total data dikumpulkan: 1,031


,No,Abstrak
0,1,"Satiyah, Pengaruh Faktor-faktor Pelatihan dan ..."
1,2,Tujuan penelitian ini adalah untuk mengetahui ...
2,3,
3,4,Aplikasi nyata pemanfaatan teknologi informasi...
4,5,"Penelitian ini menggunakan metode kuantitatif,..."
...,...,...
1026,1027,Penelitian ini bertujuan untuk mengetahui perh...
1027,1028,"Uswatun Hasanah, 160211100291, Pengaruh Pelati..."
1028,1029,Tujuan dari penelitian ini adalah untuk menget...
1029,1030,Penelitian ini bertujuan: (1) Untuk mengetahui...


### **1. Pembersihan Teks**

**Pembersihan teks** adalah tahap preprocessing untuk menormalkan data teks agar konsisten dan siap dianalisis. Proses ini meliputi: mengubah teks ke huruf kecil (lowercasing), menghapus angka, menghilangkan tanda baca, dan membersihkan spasi berlebih. Tujuannya adalah mengurangi variasi yang tidak perlu dan memfokuskan analisis pada konten tekstual yang bermakna.

In [3]:
# Pembersihan teks
import re
import pandas as pd

def clean_text(text):
    # Ubah ke huruf kecil
    text = text.lower()
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan teks
cleaned_corpus = [clean_text(text) for text in corpus]

# Tampilkan hasil pembersihan teks dalam format DataFrame
comparison_df = pd.DataFrame({
    'abstrak_asli': corpus[:10],
    'abstrak_bersih': cleaned_corpus[:10]
})

print("\nHasil Pembersihan Teks:")
comparison_df


Hasil Pembersihan Teks:


,abstrak_asli,abstrak_bersih
0,"Satiyah, Pengaruh Faktor-faktor Pelatihan dan ...",satiyah pengaruh faktorfaktor pelatihan dan pe...
1,Tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian ini adalah untuk mengetahui ...
2,,
3,Aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
4,"Penelitian ini menggunakan metode kuantitatif,...",penelitian ini menggunakan metode kuantitatif ...
5,"Aththaariq, Pengaruh Kompetensi Dosen Terhadap...",aththaariq pengaruh kompetensi dosen terhadap ...
6,"Haryono Arifin, Pengaruh Perilaku Konsumen Ter...",haryono arifin pengaruh perilaku konsumen terh...
7,"Dharma Abidin Syah,Kesimpulan: (1) Terdapat pe...",dharma abidin syahkesimpulan terdapat pengaruh...
8,Tujuan penelitian ini adalah untuk mengidentif...,tujuan penelitian ini adalah untuk mengidentif...
9,Hasil dari penelitian ini dari perhitungan Cre...,hasil dari penelitian ini dari perhitungan cre...


### **2. Tokenisasi**

**Tokenisasi** adalah proses memecah teks menjadi unit-unit yang lebih kecil seperti kata atau token. Pada tahap ini, setiap kalimat dalam abstrak dipecah menjadi kata-kata individual menggunakan `word_tokenize` dari NLTK. Contoh: kalimat "Penelitian ini menganalisis data" → ['Penelitian', 'ini', 'menganalisis', 'data']

In [4]:
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

nltk.download('punkt_tab')

# Tokenisasi untuk PTA
# Buat DataFrame dari corpus yang sudah dibersihkan
pta_df = pd.DataFrame({
    'abstrak_id_clean': cleaned_corpus
})

pta_df["abstrak_id_tokens"] = pta_df["abstrak_id_clean"].apply(word_tokenize)

print("\nPTA (abstrak_id_tokens):")
pta_df[["abstrak_id_clean", "abstrak_id_tokens"]].head(10)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



PTA (abstrak_id_tokens):


,abstrak_id_clean,abstrak_id_tokens
0,satiyah pengaruh faktorfaktor pelatihan dan pe...,"[satiyah, pengaruh, faktorfaktor, pelatihan, d..."
1,tujuan penelitian ini adalah untuk mengetahui ...,"[tujuan, penelitian, ini, adalah, untuk, menge..."
2,,[]
3,aplikasi nyata pemanfaatan teknologi informasi...,"[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,penelitian ini menggunakan metode kuantitatif ...,"[penelitian, ini, menggunakan, metode, kuantit..."
5,aththaariq pengaruh kompetensi dosen terhadap ...,"[aththaariq, pengaruh, kompetensi, dosen, terh..."
6,haryono arifin pengaruh perilaku konsumen terh...,"[haryono, arifin, pengaruh, perilaku, konsumen..."
7,dharma abidin syahkesimpulan terdapat pengaruh...,"[dharma, abidin, syahkesimpulan, terdapat, pen..."
8,tujuan penelitian ini adalah untuk mengidentif...,"[tujuan, penelitian, ini, adalah, untuk, mengi..."
9,hasil dari penelitian ini dari perhitungan cre...,"[hasil, dari, penelitian, ini, dari, perhitung..."


### **3. Penghapusan Kata Umum (Stop Words)**

**Stop Words** adalah kata-kata umum yang sering muncul dalam bahasa namun tidak memberikan makna signifikan untuk analisis teks, seperti "dan", "atau", "di", "ke", "yang". Penghapusan stopwords bertujuan untuk mengurangi noise dan fokus pada kata-kata yang lebih bermakna untuk analisis. NLTK menyediakan daftar stopwords bahasa Indonesia yang sudah siap pakai.

In [5]:
from nltk.corpus import stopwords
import nltk

# Download stopwords untuk bahasa Indonesia
nltk.download('stopwords')

# Stopwords untuk bahasa Indonesia
stop_words_id = set(stopwords.words('indonesian'))

# Filter stopwords di PTA
pta_df["abstrak_id_filtered"] = pta_df["abstrak_id_tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words_id]
)

print("\nPTA (abstrak_id_filtered):")
pta_df[["abstrak_id_tokens", "abstrak_id_filtered"]].head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



PTA (abstrak_id_filtered):


,abstrak_id_tokens,abstrak_id_filtered
0,"[satiyah, pengaruh, faktorfaktor, pelatihan, d...","[satiyah, pengaruh, faktorfaktor, pelatihan, p..."
1,"[tujuan, penelitian, ini, adalah, untuk, menge...","[tujuan, penelitian, persepsi, brand, associat..."
2,[],[]
3,"[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,"[penelitian, ini, menggunakan, metode, kuantit...","[penelitian, metode, kuantitatif, menekankan, ..."
5,"[aththaariq, pengaruh, kompetensi, dosen, terh...","[aththaariq, pengaruh, kompetensi, dosen, kine..."
6,"[haryono, arifin, pengaruh, perilaku, konsumen...","[haryono, arifin, pengaruh, perilaku, konsumen..."
7,"[dharma, abidin, syahkesimpulan, terdapat, pen...","[dharma, abidin, syahkesimpulan, pengaruh, sig..."
8,"[tujuan, penelitian, ini, adalah, untuk, mengi...","[tujuan, penelitian, mengidentifikasi, variabe..."
9,"[hasil, dari, penelitian, ini, dari, perhitung...","[hasil, penelitian, perhitungan, credit, risk,..."


### **4. Stemming dan Lematisasi**

**Stemming** adalah proses mengubah kata-kata menjadi bentuk dasar (root/stem) dengan menghilangkan imbuhan seperti awalan, akhiran, dan sisipan. Contoh: "penelitian" → "teliti", "menganalisis" → "analisis". **Lematisasi** serupa dengan stemming namun menghasilkan kata dasar yang lebih bermakna secara linguistik. Pada kode ini menggunakan library Sastrawi untuk stemming bahasa Indonesia dan TF-IDF untuk mengukur kepentingan kata dalam dokumen.

In [7]:
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Test stemming dengan satu dokumen
input_stemm = str(pta_df["abstrak_id_filtered"].iloc[0])
hasil_stemm = stemmer.stem(input_stemm)
print(hasil_stemm)

# Stemming untuk semua dokumen
hasil_stemm = []
for doc in pta_df["abstrak_id_filtered"]:
    stemmed_doc = [stemmer.stem(word) for word in doc]
    hasil_stemm.append(stemmed_doc)

# Konversi hasil stemming ke string untuk TF-IDF
stemmed_texts = [' '.join(doc) for doc in hasil_stemm]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(stemmed_texts)
print(x)

satiyah pengaruh faktorfaktor latih kembang produktivitas kerja dinas laut ikan bangkal bawah bimbing drahjsanugrahini irawatimm helm buyung auliasstsemmt upaya tingkat produktivitas kerja mudah salah usaha produktivitas tingkat terap program latih kembang sumber daya manusia sdm laksana instansi produktivitas capai tingkat mampu pegawai efektif efisien latih pengembnagan harap pegawai sesuai kebutuhankebutuhan sikap tingkah laku terampil tahu sesuai tuntut ubah latih kembang pegawai dukung cipta suasana kerja kondusif instansi produktivitas kerja tingkat tuju teliti pengaruh faktorfaktor latih kembang produktivitas kerja dinas laut ikan bangkal ukur menganalisa hubung variabel teliti dekat observasional analitik amat langsung responden sebar kuisioner analis teliti teliti populasi responden sampel olah spss versi analis metode statistik metode non probality sampling simple random sampling simpul teliti a faktorfaktor latih kembang beda individu pegawai x hubung analisis jabat x motiva

### **5. Analisis Frekuensi Kata**

**Analisis Frekuensi** adalah proses menghitung seberapa sering setiap kata muncul dalam corpus setelah melalui tahap preprocessing. Menggunakan `FreqDist` dari NLTK untuk menghitung distribusi frekuensi kata dan mengidentifikasi kata-kata yang paling sering muncul. Hasil analisis ini disimpan dalam format CSV untuk analisis lebih lanjut dan memberikan insight tentang tema-tema utama dalam dokumen.

In [8]:
from nltk import FreqDist
import pandas as pd

# Gabungkan semua kata dari hasil stemming
all_words = []
for doc in hasil_stemm:
    all_words.extend(doc)

fdist = FreqDist(all_words)

frequency_df = pd.DataFrame(
    fdist.most_common(),
    columns=['Word', 'Frequency']
)

frequency_df.to_csv('Management_dataHasilPreprocessing.csv', index=False, encoding='utf-8-sig')
print("File disimpan sebagai Management_dataHasilPreprocessing.csv")

File disimpan sebagai Management_dataHasilPreprocessing.csv
